### Sales Analysis 

In [18]:
# Import Python lib
import pandas as pd
import numpy as np
import seaborn as sns

# load datafile 
df= pd.read_csv("AusApparalSales4thQrt2020.csv")

## 1. Data Wrangling

### 1.a. check missing values

In [19]:
df.isna().sum()
## result => no missing data

Date     0
Time     0
State    0
Group    0
Unit     0
Sales    0
dtype: int64

### 1.b Treating missing or incorrect data

Since there was no missing data, there is no corection required.
However, for this data, since it's sales record for stores, I would drop the rows with missing data.  If it's categorical data, I would correct the data with Unknown.

In [20]:
# checking data types
df.dtypes

Date     object
Time     object
State    object
Group    object
Unit      int64
Sales     int64
dtype: object

In [21]:
#date column is objet, convert it to date column
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.dtypes
# The Date field has been confirmed to the correct type.

Date     datetime64[ns]
Time             object
State            object
Group            object
Unit              int64
Sales             int64
dtype: object

In [22]:
# Find duplicate rows based on all columns
duplicates = df[df.duplicated(keep=False)]

# Count occurrences of each duplicate row
duplicate_counts=duplicates.groupby(list(duplicates.columns)).size().reset_index(name='count')
print(duplicate_counts)

## => result no duplicate found

Empty DataFrame
Columns: [Date, Time, State, Group, Unit, Sales, count]
Index: []


### 1.C Choose a sutable data wrangling technique
- either data standardization or normalization. Execute the preferred normalization method and present the resulting data. 
- Normalization is the preferred approach for this problem (Min-Max or Standard normalization)

### 1.d Share your insights regarding the application of the GroupBy() function for either data chunking or merging, and offer a recommendation based on your analysis.

## 2. Data Analysis

### 2.a.  Perform descriptive statistical analysis on the data in the Sales and Unit columns. Utilize techniques such as mean, median, mode, and standard deviation for this analysis.

In [28]:
## 2. Data Analysis
df.head()

,Date,Time,State,Group,Unit,Sales
0,2020-10-01,Morning,WA,Kids,8,20000
1,2020-10-01,Morning,WA,Men,8,20000
2,2020-10-01,Morning,WA,Women,4,10000
3,2020-10-01,Morning,WA,Seniors,15,37500
4,2020-10-01,Afternoon,WA,Kids,3,7500


### 2.b Identify the group with the highest sales and the group with the lowest sales based on the data provided.

### 2.c. Identify the group with the highest and lowest sales based on the data provided.

### 2.d Generate weekly, monthly, and quarterly reports to document and present the results of the analysis conducted. (Use suitable libraries such as NumPy, Pandas, and SciPy for performing the analysis.)

## 3. Data visualization

### 3.a. 
- Use suitable data visualization libraries to construct a dashboard for the head of sales and marketing. The dashboard should encompass key parameters:
State-wise sales analysis for different demographic groups (kids, women, men, and seniors).
- Group-wise sales analysis (Kids, Women, Men, and Seniors) across various states.
- Time-of-the-day analysis: Identify peak and off-peak sales periods to facilitate strategic planning for S&M teams. This information aids in designing programs like hyper-personalization and Next Best Offers to enhance sales.

### 3.b Ensure the visualization is clear and accessible for effective decision-making by the head of sales and marketing (S&M).
The dashboard must contain daily, weekly, monthly, and quarterly charts.
(Any visualization library can be used for this purpose. However, since statistical analysis is being done, Seaborn is preferred.)

### 3.c.

Include your recommendation and indicate why you are choosing the recommended visualization package.

## 4. Report generation

### 4.a

Use JupyterLab Notebook for generating reports, which includes tasks such as data wrangling, analysis, and visualization. Please note that JupyterLab enables you to integrate code seamlessly with graphs and plots


### 4.b
Use Markdown in suitable places while presenting your report. 

### 4.c
Use suitable graphs, plots, and analysis reports in the report, along with recommendations. Note that various aspects of analysis require different graphs and plots.
- Use a box plot for descriptive statistics.
- Use the Seaborn distribution plot for any other statistical plotting.